In [1]:
# https://www.kaggle.com/hhyun2/hubmap-pytorch-gpu-training/comments

import sys
import os
sys.path.append(os.path.join(str(os.path.abspath('')), "..", "..", ".."))

from src.utils.losses import DiceLoss, DiceBCELoss
from src.utils.dice_coef import DiceCoef
from src.utils.set_seed import seed_torch
from src.utils.transforms import get_transform

from sklearn.model_selection import GroupKFold
import torch
from torch import nn
import torchvision
import cv2
import os
import numpy as np
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from scipy.ndimage.interpolation import zoom
import albumentations as A
from torch.nn import functional as F
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import sys
import time
import random

import segmentation_models_pytorch as smp
from segmentation_models_pytorch import Unet, FPN, UnetPlusPlus

In [2]:
class CFG:
    data = 512 #256
    debug=False
    apex=False
    print_freq=100
    num_workers=4
    img_size=512 # appropriate input size for encoder 
    epoch=20 # Change epochs
    criterion= 'DiceBCELoss' #'DiceBCELoss' # ['DiceLoss', 'Hausdorff', 'Lovasz']
    lr=1e-4
    min_lr=1e-6
    batch_size=10
    weight_decay=1e-5
    seed=251096
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    inference=False
    optimizer = 'Adam'
    T_0=10
    # N=5 
    # M=9
    T_max=10
    factor=0.1
    patience=1
    eps=1e-8
    smoothing=1
    train_transform_type = 'strong'
    valid_transform_type = 'val'

In [3]:
# TEST_IMAGES = ['b9a3865fc', '2f6ecfcdf', '26dc41664']
seed_torch(seed=CFG.seed)
train_transform = get_transform(img_size=CFG.img_size, transform_type=CFG.train_transform_type)
valid_transform = get_transform(img_size=CFG.img_size, transform_type=CFG.valid_transform_type)
RESIZED_DATA_DIR = '../../../../../hdd/storage/kidney_segmentation/working_kaggle/upd_resized/'

In [4]:
class HuBMAPDataset(Dataset):
    def __init__(self,df, train='train', transform=True):
        ids = df.id.values
        #kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
        #ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
#         if CFG.data==1024:
#             self.fnames = [fname for fname in os.listdir(RESIZED_DATA_DIR + '1024*1024/train/') if fname.split('_')[0] in ids]
#         elif CFG.data==512:
#             self.fnames = [fname for fname in os.listdir(RESIZED_DATA_DIR + '512*512/train/') if fname.split('_')[0] in ids]
#         elif CFG.data==256:
#             self.fnames = [fname for fname in os.listdir(RESIZED_DATA_DIR + '256*256/train/') if fname.split('_')[0] in ids]
        self.fnames = ids
#         print(self.fnames)
        self.train = train
        self.transform = transform
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fname = self.fnames[idx]
        if CFG.data==1024:
            img = cv2.cvtColor(cv2.imread(os.path.join(RESIZED_DATA_DIR + '1024*1024/train/',fname)), cv2.COLOR_BGR2RGB)
            mask = cv2.imread(os.path.join(RESIZED_DATA_DIR + '1024*1024/masks',fname),cv2.IMREAD_GRAYSCALE)
        elif CFG.data==512:
            img = cv2.cvtColor(cv2.imread(os.path.join(RESIZED_DATA_DIR + '512*512/train/',fname)), cv2.COLOR_BGR2RGB)
            mask = cv2.imread(os.path.join(RESIZED_DATA_DIR + '512*512/masks',fname),cv2.IMREAD_GRAYSCALE)
        elif CFG.data==256:
            img = cv2.cvtColor(cv2.imread(os.path.join(RESIZED_DATA_DIR + '256*256/train/',fname)), cv2.COLOR_BGR2RGB)
            mask = cv2.imread(os.path.join(RESIZED_DATA_DIR + '256*256/masks',fname),cv2.IMREAD_GRAYSCALE)
        
        if self.train == 'train':
            if self.transform == True:
                augmented = train_transform(image=img,mask=mask)
                img,mask = augmented['image'],augmented['mask']
                    
        elif self.train == 'val':
            transformed = valid_transform(image=img,mask=mask)
            img,mask = transformed['image'],transformed['mask']
            
        img = img.type('torch.FloatTensor')
        img = img/255
        mask = mask.type('torch.FloatTensor')

        return img, mask
          


In [5]:
train_df = pd.read_csv("../../../../../hdd/storage/kidney_segmentation/input_kaggle/train.csv")
train_df

,id,encoding
0,2f6ecfcdf,296084587 4 296115835 6 296115859 14 296147109...
1,8242609fa,96909968 56 96941265 60 96972563 64 97003861 6...
2,aaa6a05cc,30989109 59 31007591 64 31026074 68 31044556 7...
3,cb2d976f4,78144363 5 78179297 15 78214231 25 78249165 35...
4,b9a3865fc,61271840 4 61303134 13 61334428 22 61365722 30...
5,b2dc8411c,56157731 21 56172571 45 56187411 51 56202252 5...
6,0486052bb,101676003 6 101701785 8 101727568 9 101753351 ...
7,e79de561c,7334642 14 7350821 41 7367001 67 7383180 82 73...
8,095bf7a1f,113277795 21 113315936 53 113354083 87 1133922...
9,54f2eec69,124967057 36 124997425 109 125027828 147 12505...


In [6]:
if CFG.data==1024:
    directory_list = os.listdir(RESIZED_DATA_DIR + '1024*1024/train')
elif CFG.data==512:
    directory_list = os.listdir(RESIZED_DATA_DIR + '512*512/train')
elif CFG.data==256:
    directory_list = os.listdir(RESIZED_DATA_DIR + '256*256/train')
# directory_list = [fnames.split('_')[0] for fnames in directory_list]
# dir_df = pd.DataFrame(directory_list, columns=['id'])
# dir_df

In [7]:
if CFG.criterion == 'DiceBCELoss':
    criterion = DiceBCELoss()
elif CFG.criterion == 'DiceLoss':
    criterion = DiceLoss()
elif CFG.criterion == 'Hausdorff':
    criterion = Hausdorff_loss()
elif CFG.criterion == 'Lovasz':
    criterion = Lovasz_loss()

In [8]:
def HuBMAPLoss(images, targets, model, device, loss_func=criterion):
    model.to(device)
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    loss_func = loss_func
    loss = loss_func(outputs, targets)
    return loss, outputs

def HuBMAPMetric(outputs, targets):
#     model.to(device)
#     images = images.to(device)
#     targets = targets.to(device)
#     outputs = model(images)
    criterion = DiceCoef()
    metric = criterion(outputs, targets)
    return metric

In [9]:
def train_one_epoch(epoch, model, device, optimizer, trainloader):
    model.train()
    t = time.time()
    total_loss = 0
    total_metric = 0
    total_metric_new = 0
    for step, (images, targets) in enumerate(trainloader):
        optimizer.zero_grad()
        loss, outputs = HuBMAPLoss(images, targets, model, device)
        loss.backward()
#         if ((step+1)%4==0 or (step+1)==len(trainloader)):
        optimizer.step()        
        loss = loss.detach().item()
        total_loss += loss
#         print("ou:", outputs)
#         print("tar", targets)
        metric, metric_new = HuBMAPMetric(outputs.cpu(), targets)
        total_metric += metric
        total_metric_new += metric_new
        if ((step+1)%10==0 or (step+1)==len(trainloader)):
            print(
                    f'epoch {epoch} train step {step+1}/{len(trainloader)}, ' + \
                    f'loss: {total_loss/len(trainloader):.4f}, ' + \
                    f'dice_coef: {total_metric/len(trainloader):.4f}, ' + \
                    f'dice_coef_new: {total_metric_new/len(trainloader):.4f}, ' + \
                    f'time: {(time.time() - t):.4f}', end= '\r' if (step + 1) != len(trainloader) else '\n'
                )
    
        
        
        
def valid_one_epoch(epoch, model, device, optimizer, validloader):
    model.eval()
    t = time.time()
    total_loss = 0
    total_metric = 0
    total_metric_new = 0
    for step, (images, targets) in enumerate(validloader):
        loss, outputs = HuBMAPLoss(images, targets, model, device)
        loss = loss.detach().item()
        total_loss += loss
        metric, metric_new = HuBMAPMetric(outputs.cpu(), targets)
        total_metric += metric
        total_metric_new += metric_new
#         if ((step+1)%4==0 or (step+1)==len(validloader)):
#             scheduler.step(total_loss/len(validloader))
        if ((step+1)%10==0 or (step+1)==len(validloader)):
            print(
                    f'epoch {epoch} valid step {step+1}/{len(validloader)}, ' + \
                    f'loss: {total_loss/len(validloader):.4f}, ' + \
                    f'dice_coef: {total_metric/len(validloader):.4f}, ' + \
                    f'dice_coef_new: {total_metric_new/len(validloader):.4f}, ' + \
                    f'time: {(time.time() - t):.4f}', end= '\r' if (step + 1) != len(validloader) else '\n'
                )
    return total_loss/len(validloader)

In [10]:
dir_df = pd.read_csv('dir_df.csv')
dir_df

,id,mask_density,bin_label,Folds
0,afa5e8098_0583.png,0,0.0,0
1,0486052bb_0395.png,2010,9.0,3
2,afa5e8098_0981.png,0,0.0,0
3,afa5e8098_0591.png,0,0.0,0
4,095bf7a1f_0982.png,2363,10.0,4
...,...,...,...,...
10260,b2dc8411c_0078.png,0,0.0,4
10261,c68fe75ea_0268.png,0,0.0,4
10262,095bf7a1f_0519.png,0,0.0,4
10263,095bf7a1f_1116.png,0,0.0,4


In [11]:
def prepare_train_valid_dataloader(df, fold):
    train_ids = df[~df.Folds.isin(fold)]
    val_ids = df[df.Folds.isin(fold)]
    
    train_ds = HuBMAPDataset(train_ids, train='train', transform=True)
    val_ds = HuBMAPDataset(val_ids, train='val', transform=True)
    train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=False, num_workers=4)
    return train_loader, val_loader

In [12]:
class HuBMAP(nn.Module):
    def __init__(self):
        super(HuBMAP, self).__init__()
        self.cnn_model = smp.Unet('efficientnet-b5', encoder_weights="imagenet", classes=1)
        self.metric = 0
        #self.cnn_model.decoder.blocks.append(self.cnn_model.decoder.blocks[-1])
        #self.cnn_model.decoder.blocks[-2] = self.cnn_model.decoder.blocks[-3]
    
    def forward(self, imgs):
        img_segs = self.cnn_model(imgs)
        return img_segs

In [13]:
CFG.patience

1

In [14]:
for fold in [0, 1, 2, 3, 4]:
    trainloader, validloader = prepare_train_valid_dataloader(dir_df, [fold])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = HuBMAP().to(device)
    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, 
                     amsgrad=False
                    )
#     scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=CFG.patience, min_lr=1e-6, factor=CFG.factor, verbose=True, eps=CFG.eps)
#         scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=7)

    num_epochs = CFG.epoch
    for epoch in range(num_epochs):
        print(epoch)
        train_one_epoch(epoch, model, device, optimizer, trainloader)
        with torch.no_grad():
            loss = valid_one_epoch(epoch, model, device, optimizer, validloader)
        scheduler.step(loss)
    torch.save(model.state_dict(),f'FOLD-{fold}-modellkjljklj.pth')

0


/home/greylord/hubmap_kidney/hubmap_venv/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch 0 train step 822/822, loss: 0.7000, dice_coef: 0.4276, dice_coef_new: 0.7418, time: 436.4009
epoch 0 valid step 206/206, loss: 0.4092, dice_coef: 0.6145, dice_coef_new: 0.8117, time: 28.7253
1
epoch 1 train step 822/822, loss: 0.2212, dice_coef: 0.8078, dice_coef_new: 0.8665, time: 444.4202
epoch 1 valid step 206/206, loss: 0.2752, dice_coef: 0.7456, dice_coef_new: 0.8448, time: 29.1647
2


KeyboardInterrupt: 

In [ ]:
dir_df.to_csv('dir_df.csv', index=False)